#  Neural Networks: Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

In [1]:
#put here your ``numero di matricola''
numero_di_matricola = 1 # COMPLETE

## Insert your ID number ("numero di matricola") below

In [2]:
#put here your ``numero di matricola''
numero_di_matricola = 1 # COMPLETE

In [3]:
#import all packages needed
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [4]:
#load the data
df = pd.read_csv('kc_house_data.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna() 

df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,3.164000e+03,3.164000e+03,3164.000000,3164.000000,3164.000000,3.164000e+03,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000
mean,4.645240e+09,5.354358e+05,3.381163,2.071903,2070.027813,1.525054e+04,1.434893,0.009798,0.244311,3.459229,7.615676,1761.252212,308.775601,1967.489254,94.668774,98077.125158,47.557868,-122.212337,1982.544564,13176.302465
std,2.854203e+09,3.809004e+05,0.895472,0.768212,920.251879,4.254457e+04,0.507792,0.098513,0.776298,0.682592,1.166324,815.934864,458.977904,28.095275,424.439427,54.172937,0.140789,0.139577,686.256670,25413.180755
min,1.000102e+06,7.500000e+04,0.000000,0.000000,380.000000,6.490000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,380.000000,0.000000,1900.000000,0.000000,98001.000000,47.177500,-122.514000,620.000000,660.000000
25%,2.199775e+09,3.150000e+05,3.000000,1.500000,1430.000000,5.453750e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1950.000000,0.000000,98032.000000,47.459575,-122.324250,1480.000000,5429.500000
50%,4.027701e+09,4.450000e+05,3.000000,2.000000,1910.000000,8.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1545.000000,0.000000,1969.000000,0.000000,98059.000000,47.572500,-122.226000,1830.000000,7873.000000
75%,7.358175e+09,6.402500e+05,4.000000,2.500000,2500.000000,1.122250e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2150.000000,600.000000,1990.000000,0.000000,98117.000000,47.680250,-122.124000,2360.000000,10408.250000
max,9.839301e+09,5.350000e+06,8.000000,6.000000,8010.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,12.000000,6720.000000,2620.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5790.000000,425581.000000


Extract input and output data. We want to predict the price by using features other than id as input.

In [5]:
Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:",m)
Y = Data[:m,2]
X = Data[:m,3:]

Amount of data: 3164


## Data Pre-Processing

We split the data into 3 parts: one will be used for training and choosing the parameters, one for choosing among different models, and one for testing. The part for training and choosing the parameters will consist of $2/3$ of all samples, the one for choosing among different models will consist of $1/6$ of all samples, while the other part consists of the remaining $1/6$-th of all samples.

In [6]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:",m_train)
print("Amount of data for validation (choosing among different models):",m_val)
print("Amount of data for test:",m_test)
from sklearn.model_selection import train_test_split

Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(X, Y, test_size=m_test/m, random_state=numero_di_matricola)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val), random_state=numero_di_matricola)

Amount of data for training and deciding parameters: 2109
Amount of data for validation (choosing among different models): 527
Amount of data for test: 528


Let's standardize the data.

In [7]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)

## Neural Networks
Let's start by learning a simple neural network with 1 hidden node.
Note: we are going to use the input parameter solver='lbfgs' and random_state=numero_di_matricola to fix the random seed (so results are reproducible).

In [10]:
#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model

NN = MLPRegressor(hidden_layer_sizes = (1,), solver = "lbfgs", random_state = numero_di_matricola)

#let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Error on training data: ", 1 - NN.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Error on validation data: ", 1 - NN.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(NN.intercepts_)

Error on training data:  0.26394829987792656
Error on validation data:  0.30404685789529706
[array([[-214.80765928],
       [ 268.96290924],
       [ 524.01594613],
       [ -60.62313943],
       [   3.61781653],
       [ 710.6640988 ],
       [ 294.47836947],
       [ 136.57678243],
       [ 814.68281475],
       [ 493.62511909],
       [ 163.29062437],
       [-581.29588365],
       [  38.20577038],
       [-203.4301266 ],
       [ 600.22940461],
       [-142.14771626],
       [ 147.44059903],
       [ -27.19275149]]), array([[141.16560883]])]
[array([3793.50667568]), array([-48.05906964])]


## Neural Networks vs Linear Models

Let's learn a linear model on the other same data and compare the results with the simple NN above.

In [12]:
from sklearn import linear_model

LR = linear_model.LinearRegression()

#fit the model on training data
LR.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Error on training data: ", 1 - LR.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Error on validation data: ", 1 - LR.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(LR.coef_)

#let's print the coefficient for the bias (i.e., the bias)
print(LR.intercept_)

Error on training data:  0.2653594216072852
Error on validation data:  0.3115400506517969
[-31303.71909156  35848.45081517  74506.78099995  -8012.41104949
    671.23713588 100205.53195594  41671.19028923  19507.84532115
 111331.50566184  69959.22677526  23468.73219785 -78236.93092911
   6535.34729956 -28197.21476235  83701.76486765 -21647.26671149
  22056.22833416  -2002.69401407]
536831.9203413766


Is there a way to make a NN network learn a linear model?

Let's first check what is the loss used by MLPRegressor...

In [13]:
#let's write the code to learn a linear model with NN: how? 

#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model

NN = MLPRegressor(hidden_layer_sizes = (1,), solver = "lbfgs", random_state = numero_di_matricola, activation = "identity")

#let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Error on training data: ", 1 - NN.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Error on validation data: ", 1 - NN.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(NN.intercepts_)

Error on training data:  0.26535942166590454
Error on validation data:  0.31153906582827584
[array([[  51.55070235],
       [ -59.02846704],
       [-122.96939596],
       [  13.19466306],
       [  -1.10694448],
       [-165.016906  ],
       [ -68.62491747],
       [ -32.12596414],
       [-183.34020234],
       [-114.96558392],
       [ -38.51551396],
       [ 128.83877577],
       [ -10.76237595],
       [  46.43749972],
       [-137.83874569],
       [  35.64844107],
       [ -36.32214078],
       [   3.29740834]]), array([[-607.24280969]])]
[array([-883.44724722]), array([365.29153421])]


Note that there is an $\ell_2$ regularization term in MLPRegressor. What about making it smaller?

In [ ]:
#COMPLETE

## More Complex NNs

Let's try more complex NN, for example increasing the number of nodes in the only hidden layer, or increasing the number of hidden layers.

Let's build a NN with 2 nodes in the only hidden layer

In [14]:
#let's build a NN with 2 nodes in the only hidden layer

#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model

NN = MLPRegressor(hidden_layer_sizes = (2,), solver = "lbfgs", random_state = numero_di_matricola)

#let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Error on training data: ", 1 - NN.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Error on validation data: ", 1 - NN.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(NN.intercepts_)

Error on training data:  0.18062109487942002
Error on validation data:  0.2071985800968661
[array([[  90.99941381,  -33.31779023],
       [ 120.42499648,   39.32679895],
       [  85.74353857,   73.03492528],
       [-261.23164773,   28.27522451],
       [ -30.58416588,   17.89847762],
       [ 198.14343088,   25.46992154],
       [  34.88372182,   37.57366671],
       [  96.94676979,   25.69592196],
       [ 312.39515229,  132.8764007 ],
       [  85.00238164,   68.82140272],
       [  19.25029521,   23.37631945],
       [-217.42668935,  -81.21134469],
       [  -3.51203035,   20.11174383],
       [-301.19459787,  -26.38167448],
       [ 305.52080917,  144.59932976],
       [-463.75030123,  -16.40475018],
       [ 194.5398277 ,   52.91459507],
       [-250.87012025,  -11.15825198]]), array([[614.56980061],
       [548.8458984 ]])]
[array([-1049.61719913,   897.58810555]), array([725.96792918])]


Let's build a NN with 5 nodes in the only hidden layer

In [15]:
#let's build a NN with 5 nodes in the only hidden layer

#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model

NN = MLPRegressor(hidden_layer_sizes = (5,), solver = "lbfgs", random_state = numero_di_matricola)

#let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Error on training data: ", 1 - NN.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Error on validation data: ", 1 - NN.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
print(NN.intercepts_)

Error on training data:  0.16260879498841674
Error on validation data:  0.2145202694399876
[array([[ -168.09486982,   143.47517422,    94.01034403,   172.70732352,
          -55.64719254],
       [  298.94903373,   397.23600398,   225.43643321,   134.16188775,
           -4.74482326],
       [ -380.96028133,   333.78677087,   227.73462878,  -306.05084185,
          287.55866552],
       [ -160.87033474,   -24.42821603,  -750.80921338,    97.3368281 ,
           74.59941455],
       [  831.28279715,  -913.77714326,   148.8889618 ,   -35.25938417,
          -95.68376385],
       [ -379.34965485,   682.70626308,   537.79964382,  -509.32700206,
          -90.85272994],
       [ -590.90249629,   663.71024863,   -18.46692083,  -455.20630629,
          203.75629121],
       [   96.51283147,   765.83586851,   168.74040587,   -44.97937112,
           33.98481024],
       [ -399.51068088,   684.1429114 ,   614.97455191,    77.34360098,
          426.63939177],
       [ -488.60649903,   574.35847

C:\Users\Giovanni\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Note that with a smaller number of iterations we had a larger error on training set but a smaller error on validation data -> "early stopping is a form of regularization"

Let's build a NN with 10 nodes in the only hidden layer

In [16]:
#let's build a NN with 10 nodes in the only hidden layer

#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model

NN = MLPRegressor(hidden_layer_sizes = (10,), solver = "lbfgs", random_state = numero_di_matricola)

#let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Error on training data: ", 1 - NN.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Error on validation data: ", 1 - NN.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#print(NN.intercepts_)

Error on training data:  0.1226812203980927
Error on validation data:  0.3028737849736476


C:\Users\Giovanni\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's build a NN with 100 nodes in the only hidden layer. Note that this is the default!

In [17]:
#let's build a NN with 100 nodes in the only hidden layer

#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model

NN = MLPRegressor(hidden_layer_sizes = (100,), solver = "lbfgs", random_state = numero_di_matricola)

#let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Error on training data: ", 1 - NN.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Error on validation data: ", 1 - NN.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#print(NN.intercepts_)

Error on training data:  0.031779439320063885
Error on validation data:  0.5431090816679929


C:\Users\Giovanni\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 1 node each

In [18]:
#let's build a NN with 2 hidden layers, 1 node each

#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model

NN = MLPRegressor(hidden_layer_sizes = (1, 1,), solver = "lbfgs", random_state = numero_di_matricola)

#let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Error on training data: ", 1 - NN.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Error on validation data: ", 1 - NN.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#print(NN.intercepts_)

Error on training data:  0.23811125681551426
Error on validation data:  0.273027867947361


C:\Users\Giovanni\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 2 nodes each

In [19]:
#let's build a NN with 2 layers, 2 nodes each

#let's load the MLPRegressor

from sklearn.neural_network import MLPRegressor

#let's define the model

NN = MLPRegressor(hidden_layer_sizes = (2, 2,), solver = "lbfgs", random_state = numero_di_matricola)

#let's learn the model on training data
NN.fit(Xtrain_scaled, Ytrain)

#let's print the error (1 - R^2) on training data
print("Error on training data: ", 1 - NN.score(Xtrain_scaled, Ytrain))

#let's print the error (1 - R^2) on validation data
print("Error on validation data: ", 1 - NN.score(Xval_scaled, Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#print(NN.intercepts_)

Error on training data:  0.21289957649687308
Error on validation data:  0.2692207014633645


C:\Users\Giovanni\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 10 nodes each

In [ ]:
#let's build a NN with 2 layers, 10 nodes each

#COMPLETE

Let's try 2 layers, 100 nodes each

In [ ]:
#let's build a NN with 2 layers, 100 nodes each

#COMPLETE

So it seems that 1 layer (and default number of iterations) works best for this dataset. Let's try 5-fold cross-validation with number of nodes in the hidden layer between 1 and 20.
Note that we use train and validation data together, since we are doing cross-validation.

In [ ]:
from sklearn.model_selection import GridSearchCV

#COMPLETE

Now let's check what is the best parameter, and compare the best NNs with the linear model (learned on train and validation) on test data.

In [ ]:
#let's print the best model according to grid search
#COMPLETE

#let's print the error 1-R^2 for the best model
#COMPLETE

Let compare the error of the best NN on train and validation and on test data.

In [ ]:
#COMPLETE

Now let's learn the linear model on train and validation, and get error (1-R^2) on train and validation and on test data.

In [ ]:
#COMPLETE

Note: MLPRegressor has several other parameters!